# 动态创建类

In [21]:
def __init__(self, ins_id):
    self.ins_id = ins_id

@classmethod
def create_ins(cls):
    cls.ins_id += 1
    return cls(cls.ins_id)

BaseCls = type("BaseCls",(),{"name":"base_cls",
                   "create_date":"2022-08-23",
                   "ins_id":0,
                   "__init__":__init__,
                   "create_ins":create_ins})

base_ins_0 = BaseCls(0)
print(base_ins_0.ins_id)
print(BaseCls.ins_id)

base_ins_1 = BaseCls.create_ins()
print(base_ins_1.ins_id)
print(BaseCls.ins_id)

0
0
1
1


# `type`元类

In [22]:
class BaseCls:
    def BaseOp(self):
        pass

print(BaseCls.__class__)
base_case = BaseCls()
print(base_case.__class__)
print(base_case.__class__.__class__)

<class 'type'>
<class '__main__.BaseCls'>
<class 'type'>


In [23]:
print(str.__class__)
print(int.__class__)

<class 'type'>
<class 'type'>


# 自定义元类

## Python3创建元类

### 函数作为metaclass

In [24]:
def upper_attrs(cls_name, cls_parents, cls_attrs):
    upper_cls_attrs = {
        attr if attr.startswith("_") else attr.upper() : v
        for attr, v in cls_attrs.items()
    }
    print(upper_cls_attrs)

    return type(cls_name, cls_parents, upper_cls_attrs)

# python3 style
class BaseCls(metaclass = upper_attrs):
    _private = "private"
    public = "public"

print(hasattr(BaseCls, "public"))
print(hasattr(BaseCls, "PUBLIC"))
print(hasattr(BaseCls, "_private"))

{'__module__': '__main__', '__qualname__': 'BaseCls', '_private': 'private', 'PUBLIC': 'public'}
False
True
True


### python类创建元类

### example_1

In [25]:
class UpperAttrs(type):
    def __new__(cls, clsname, bases, attrs):
        upper_attrs = {
            attr if attr.startswith("_") else attr.upper(): v 
            for attr, v in attrs.items()
        }
        print(f"metaclass {cls.__name__}.__new__ init")

        #return type(clsname, bases, upper_attrs) # this method do not rewrite `__new__`
        #return type.__new__(cls, clsname, bases, upper_attrs)
        # using `super()` is cleaner way
        return super().__new__(cls, clsname, bases, upper_attrs)

print("<BaseCls> implements")
class BaseCls(metaclass = UpperAttrs):
    _private = "private"
    public = "public"

print(hasattr(BaseCls, "public"))
print(hasattr(BaseCls, "PUBLIC"))
print(hasattr(BaseCls, "_private"))

<BaseCls> implements
metaclass UpperAttrs.__new__ init
False
True
True


### example_2

In [26]:
class MetaCls(type):
    def __new__(cls, name, bases, attrs):
        print("MetaCls __new__ runs")
        print("cls:{}".format(cls))
        print("attrs:{}".format(attrs))
        print("-" * 40)
        return super().__new__(cls, name, bases, attrs)
        
    def __init__(cls, name, bases, attrs):
        print("MetaCls __init__ runs")
        print("cls:{}".format(cls))
        print("attrs:{}".format(attrs))
        print("-" * 40)

        return super().__init__(name, bases, attrs)
    
    def __call__(cls, *args, **kwargs):
        print("MetaCls __call__ runs")
        print("MetaCls call attrs: {}, {}".format(args, kwargs))
        print("-" * 40)

        obj = cls.__new__(cls)
        cls.__init__(obj,*args, **kwargs)

        return obj

class SubCls(metaclass = MetaCls):
    sub_name = "sub_name"
    sub_attr = "sub_attr"
    def __init__(self, tag = "<Subcls>"):
        print("Subcls __init__ {} here".format(tag))
        print("-" * 40)

print("<Finish SubCls generation>")
print("-" * 40)

obj = SubCls(tag="<obj>")

MetaCls __new__ runs
cls:<class '__main__.MetaCls'>
attrs:{'__module__': '__main__', '__qualname__': 'SubCls', 'sub_name': 'sub_name', 'sub_attr': 'sub_attr', '__init__': <function SubCls.__init__ at 0x0000021B4B032840>}
----------------------------------------
MetaCls __init__ runs
cls:<class '__main__.SubCls'>
attrs:{'__module__': '__main__', '__qualname__': 'SubCls', 'sub_name': 'sub_name', 'sub_attr': 'sub_attr', '__init__': <function SubCls.__init__ at 0x0000021B4B032840>}
----------------------------------------
<Finish SubCls generation>
----------------------------------------
MetaCls __call__ runs
MetaCls call attrs: (), {'tag': '<obj>'}
----------------------------------------
Subcls __init__ <obj> here
----------------------------------------


## `__instancecheck__`

In [27]:
class BaseCls():
    def __instancecheck__(self, instance):
        print("cls.__instancecheck__ runs")
        return True

print(isinstance(str, BaseCls))
print(isinstance(str, BaseCls()))

False
cls.__instancecheck__ runs
True


In [28]:
class MetaType(type):
    def __instancecheck__(cls, instance):
        if hasattr(instance, "base_attr"):
            return True
        else:
            return False
    
class BaseType(metaclass = MetaType):
    def __init__(self):
        self.base_attr = None

base_type = BaseType()

class ComparedType():
    def __init__(self):
        self.base_attr = None

comp_type = ComparedType()

print(isinstance(base_type, BaseType))
print(isinstance(comp_type, BaseType))
del comp_type.base_attr
print(isinstance(comp_type, BaseType))

True
True
False


## `__subclasscheck__`

In [32]:
class MetaType(type):
    def __subclasscheck__(cls, subcls):
        if super().__subclasscheck__(subcls) and hasattr(subcls, "is_sub"):
            return True
        else:
            return False

class BaseCls(metaclass = MetaType):
    pass

class SubCls(BaseCls):
    is_sub = None

print(issubclass(SubCls, BaseCls))

True


In [31]:
hasattr(SubCls, "is_sub")

True

# 元类应用

In [30]:
class MetaModel(type):
    def __new__(cls, name, bases, attrs):
        if name != "Model":
            if attrs.get("fit",None) != None:
                if  callable(attrs.get("fit",None)):
                    return super().__new__(cls, name, bases, attrs)
        else:
            return super().__new__(cls, name, bases, attrs)
        
        raise ValueError("Model Should implement fit() method.")

class Model(metaclass = MetaModel):
    def fit_transform(self):
        pass 

try:
    class SubModel(Model):
        def fit_transform(self):
            pass
except:
    print("cannot return new cls without method `fit()`")


cannot return new cls without method `fit()`
